In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import sqlite3 as sq

In [2]:
from datetime import datetime

def log_message(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    curr_time = now.strftime(timestamp_format)
    with open('log.txt', 'a') as f:
        f.write(curr_time+"_ "+message+ '\n')

In [3]:
def extract_data(url):
    try:
        log_message('Extracting url begins')
        page = requests.get(url).content
        soup = BeautifulSoup(page, 'html.parser')
        
        tables = soup.find_all('table')
        bank_data = tables[0]
        
        rows = bank_data.find_all('tr')
        # print(rows)
        table_data = []
        for each in rows:
            data = each.find_all(['td', 'th'])
            val = [cell.text.strip() for cell in data]
            table_data.append(val)

        log_message('Extraction completed.')
        df = pd.DataFrame(table_data[1:], columns=table_data[0])
        return df
    except Exception as e:
        log_message('Extraction failed. '+str(e))



In [4]:
url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
df = extract_data(url)
df

,Rank,Bank name,Market cap(US$ billion)
0,1,JPMorgan Chase,432.92
1,2,Bank of America,231.52
2,3,Industrial and Commercial Bank of China,194.56
3,4,Agricultural Bank of China,160.68
4,5,HDFC Bank,157.91
5,6,Wells Fargo,155.87
6,7,HSBC Holdings PLC,148.90
7,8,Morgan Stanley,140.83
8,9,China Construction Bank,139.82
9,10,Bank of China,136.81


In [5]:
convert_usd = lambda x, conv_value: [round(float(each) * conv_value, 2) for each in x['Market cap(US$ billion)']]

def transform_table(table):
    try:
        log_message('Transform operation begins')
        df['market_caps_GBP'] = convert_usd(table, .76)
        df['market_caps_EUR'] = convert_usd(table, .91)
        df['market_caps_INR'] = convert_usd(table, 83.97)
        log_message('Transform operation completed')
        return table  
    except Exception as e:
        log_message('Transform failed. '+str(e))

In [6]:
new_df = transform_table(df)
new_df

,Rank,Bank name,Market cap(US$ billion),market_caps_GBP,market_caps_EUR,market_caps_INR
0,1,JPMorgan Chase,432.92,329.02,393.96,36352.29
1,2,Bank of America,231.52,175.96,210.68,19440.73
2,3,Industrial and Commercial Bank of China,194.56,147.87,177.05,16337.20
3,4,Agricultural Bank of China,160.68,122.12,146.22,13492.30
4,5,HDFC Bank,157.91,120.01,143.70,13259.70
5,6,Wells Fargo,155.87,118.46,141.84,13088.40
6,7,HSBC Holdings PLC,148.90,113.16,135.50,12503.13
7,8,Morgan Stanley,140.83,107.03,128.16,11825.50
8,9,China Construction Bank,139.82,106.26,127.24,11740.69
9,10,Bank of China,136.81,103.98,124.50,11487.94


In [7]:
def load_to_csv(table):
    log_message('Save operation begins')
    path = './bank_market_caps.csv'
    table.to_csv(path)
    log_message('Save operation completed')

def load_to_db(table):
    try:
        log_message('Save operation of database table begins')
        database_name = 'bank_data.db'
        table_name = 'banks'
        sql_conn = sq.connect(database_name)
    
        table.to_sql(table_name, sql_conn, if_exists='replace', index=False)
        log_message('Save operation of database table completed')
    except Exception as e:
        log_message('Save operation of database failed. '+str(e))

In [8]:
load_to_csv(df)
# save_result_sql(df)

In [9]:
url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
# bankDF = extract_data(url)

page = requests.get(url).content
soup = BeautifulSoup(page, 'html.parser')

tables = soup.find_all('table')

# print(tables[0])
bank_data = tables[0]
rows = bank_data.find_all('tr')

In [10]:
table_data = []
for each in rows:
    data = each.find_all(['td', 'th'])
    # print(data)

    # print(data[1].text)
    val = [cell.text.strip() for cell in data]
    # print(val)
    table_data.append(val)
table_data

[['Rank', 'Bank name', 'Market cap(US$ billion)'],
 ['1', 'JPMorgan Chase', '432.92'],
 ['2', 'Bank of America', '231.52'],
 ['3', 'Industrial and Commercial Bank of China', '194.56'],
 ['4', 'Agricultural Bank of China', '160.68'],
 ['5', 'HDFC Bank', '157.91'],
 ['6', 'Wells Fargo', '155.87'],
 ['7', 'HSBC Holdings PLC', '148.90'],
 ['8', 'Morgan Stanley', '140.83'],
 ['9', 'China Construction Bank', '139.82'],
 ['10', 'Bank of China', '136.81']]

In [11]:
table_data[1:]

[['1', 'JPMorgan Chase', '432.92'],
 ['2', 'Bank of America', '231.52'],
 ['3', 'Industrial and Commercial Bank of China', '194.56'],
 ['4', 'Agricultural Bank of China', '160.68'],
 ['5', 'HDFC Bank', '157.91'],
 ['6', 'Wells Fargo', '155.87'],
 ['7', 'HSBC Holdings PLC', '148.90'],
 ['8', 'Morgan Stanley', '140.83'],
 ['9', 'China Construction Bank', '139.82'],
 ['10', 'Bank of China', '136.81']]

In [12]:
df = pd.DataFrame(table_data[1:], columns=table_data[0])
df

,Rank,Bank name,Market cap(US$ billion)
0,1,JPMorgan Chase,432.92
1,2,Bank of America,231.52
2,3,Industrial and Commercial Bank of China,194.56
3,4,Agricultural Bank of China,160.68
4,5,HDFC Bank,157.91
5,6,Wells Fargo,155.87
6,7,HSBC Holdings PLC,148.90
7,8,Morgan Stanley,140.83
8,9,China Construction Bank,139.82
9,10,Bank of China,136.81


In [19]:
df['market_caps_GBP'] = [round(float(each) * .76, 2) for each in df['Market cap(US$ billion)']]
df


,Rank,Bank name,Market cap(US$ billion),market_caps_GBP
0,1,JPMorgan Chase,432.92,329.02
1,2,Bank of America,231.52,175.96
2,3,Industrial and Commercial Bank of China,194.56,147.87
3,4,Agricultural Bank of China,160.68,122.12
4,5,HDFC Bank,157.91,120.01
5,6,Wells Fargo,155.87,118.46
6,7,HSBC Holdings PLC,148.90,113.16
7,8,Morgan Stanley,140.83,107.03
8,9,China Construction Bank,139.82,106.26
9,10,Bank of China,136.81,103.98


In [14]:
def save_result_sql(table):
    try:
        log_message('Save operation of database table begins')
        database_name = 'bank_data.db'
        table_name = 'banks'
        sql_conn = sq.connect(database_name)
    
        table.to_sql(table_name, sql_conn, if_exists='replace', index=False)
        log_message('Save operation of database table completed')
    except Exception as e:
        log_message('Save operation of database failed. '+str(e))

In [15]:
save_result_sql(df)

In [16]:
def run_queries(query):
    try:
        log_message('Running query operation begins')
        database_name = 'bank_data.db'
        sql_conn = sq.connect(database_name)
        table = pd.read_sql(query, sql_conn)
        log_message('Running query operation completed')
        return table
    except Exception as e:
        log_message('Running query operation failed. '+str(e))

In [17]:
q1 = 'select * from banks where "Rank"==1;'
q2 = 'select * from banks where "Rank"==5;'
q3 = 'select avg("market_caps_GBP") from banks;'

out1 = run_queries(q1)
out2 = run_queries(q2)
out3 = run_queries(q3)

print(out1, '\n')
print(out2, '\n')
print(out3)

  Rank       Bank name Market cap(US$ billion)  market_caps_GBP
0    1  JPMorgan Chase                  432.92           329.02 

  Rank  Bank name Market cap(US$ billion)  market_caps_GBP
0    5  HDFC Bank                  157.91           120.01 

   avg("market_caps_GBP")
0                 144.387
